# Criando uma App no Twitter

## Acessar o endereço abaixo e criar uma App: https://apps.twitter.com/

Criar login, senha e logar

Criar uma nova App clicando em Create New App

Definir os detalhes da aplicação: nome, descrição, website, etc

**No menu "Keys and Tokens" gerar as chaves da App para usar na configuração do Flume e substituir no arquivo twitterAgent.conf abaixo.**

# Configurando o agent, source, channel e sink

**Agent**: Apenas um agente chamado *TwitterAgent*

**Source**: Twitter

**Channel**: Memória

**Sink**: Registra os dados no HDFS

In [1]:
!cat twitterAgent.conf

#Nome dos componentes do agente
TwitterAgent.sources = Twitter
TwitterAgent.channels = MemChannel
TwitterAgent.sinks = HDFS

#Configuração do Source
TwitterAgent.sources.Twitter.type = com.cloudera.flume.source.TwitterSource
TwitterAgent.sources.Twitter.consumerKey = UlUqG3hE7gEPxIyxxidbv9tS2
TwitterAgent.sources.Twitter.consumerSecret = y4tx7Wrblrcm8zsiHddYC7mE79rM5fgtHoVfi1IY5TXX7315mz
TwitterAgent.sources.Twitter.accessToken = 1046705580-nJYZLCvR34l40tUwT1cqxqpF96yymjrhHibQCz3
TwitterAgent.sources.Twitter.accessTokenSecret = kLkEE5JdvwKFNSrFBCXaxymSOBzmaBJJqL19CpB1rKmpA
TwitterAgent.sources.Twitter.keywords = #hadoop, #flume, #bigdata

#Configuração do Sink
TwitterAgent.sinks.HDFS.type = hdfs
TwitterAgent.sinks.HDFS.hdfs.path = hdfs://localhost:9000/user/lucasbatista
TwitterAgent.sinks.HDFS.hdfs.fileType = DataStream
TwitterAgent.sinks.HDFS.hdfs.writeFormat = Text

#number of events written to file before it is flushed to HDFS
TwitterAgent.sinks.HDFS.hdfs.batchSize = 50 

#File size

## Nota: 
Usando as configurações descritas no site do Flume - **TwitterAgent.sources.Twitter.type = org.apache.flume.source.twitter.TwitterSource** - na configuração do source, o arquivo é gerado com caracteres ilegíveis. Assim, iremos utilizar **TwitterAgent.sources.Twitter.type = com.cloudera.flume.source.TwitterSource** e temos que copiar o arquivo flume-sources-1.0-SNAPSHOT.jar para a pasta do Flume para o seu correto funcionamento.

In [2]:
!cp resources/flume-sources-1.0-SNAPSHOT.jar ~/resources/local/flume-${FLUME_VERSION}/lib

# Abrir um terminal e iniciar o FlumeAgent

Espere um tempo para o download de alguns tweets
``` bash
flume-ng agent --conf conf --conf-file twitterAgent.conf --name TwitterAgent -Dflume.looger=INFO,console
``` 

# TrendTopics com MapReduce
Criando um rank de hashtags

In [7]:
!python mrjob-hashtag-rank.py -r hadoop --hadoop-streaming-jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-2.9.2.jar local-home/git/twitter-hashtag-graph/FlumeData.*

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Traceback (most recent call last):
  File "mrjob-hashtag-rank.py", line 30, in <module>
    RankHashtag.run()
  File "/srv/conda/envs/notebook/lib/python3.7/site-packages/mrjob/job.py", line 446, in run
    mr_job.execute()
  File "/srv/conda/envs/notebook/lib/python3.7/site-packages/mrjob/job.py", line 473, in execute
    super(MRJob, self).execute()
  File "/srv/conda/envs/notebook/lib/python3.7/site-packages/mrjob/launch.py", line 202, in execute
    self.run_job()
  File "/srv/conda/envs/notebook/lib/python3.7/site-packages/mrjob/launch.py", line 247, in run_job
    runner.run()
  File "/srv/conda/envs/notebook/lib/python3.7/site-packages/mrjob/runner.py", line 505, in run
    self._check_input_paths()
  File "/srv/conda/envs/notebook/lib/python3.7/site-packages/mrjob/runner.py", line 1157, in _check_input_paths
    self._check_input_path(path)
  File "/srv/conda/envs/notebook/lib/python3.7/

# Criando grafo de Hashtags e usuários

In [ ]:
from py2neo import Graph
graph = Graph("bolt://127.0.0.1:7687")
graph.delete_all()

print("Number of nodes: {} ".format(len(graph.nodes)))
print("Number of relationships: {} ".format(len(graph.nodes)))

In [ ]:
!python twitterMRJob.py -r hadoop --hadoop-streaming-jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-2.9.2.jar hdfs:///user/lucasbatista/FlumeData.*

In [ ]:
import sys
sys.path.insert(0, 'resources/python/')

from graphdrawer.visgraph import draw

options = {"User": "id", "Hashtag": "hashtag"}
draw(graph, options, physics=True)